In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

> Basic data analysis and data understanding

In [2]:
df = pd.read_csv("Earnings Manipulation 220.csv")

In [3]:
df.shape

(220, 11)

In [4]:
df.head()

,Company Name,Year Ending,DSRI,GMI,AQI,SGI,DEPI,SGAI,ACCR,LEVI,MANIPULATOR
0,1,31-03-2014,1.624742,1.128927,7.185053,0.366211,1.381519,1.624145,-0.166809,1.161082,1
1,2,31-03-2009,1.000000,1.606492,1.004988,13.081433,0.400000,5.198207,0.060475,0.986732,1
2,3,31-03-2008,1.000000,1.015607,1.241389,1.475018,1.169353,0.647671,0.036732,1.264305,1
3,4,31-03-2014,1.486239,1.000000,0.465535,0.672840,2.000000,0.092890,0.273434,0.680975,1
4,5,31-03-2010,1.000000,1.369038,0.637112,0.861346,1.454676,1.741460,0.123048,0.939047,1


In [5]:
df.describe()

,Company Name,DSRI,GMI,AQI,SGI,DEPI,SGAI,ACCR,LEVI,MANIPULATOR
count,220.000000,220.000000,220.000000,220.000000,220.000000,220.000000,220.000000,220.000000,220.000000,220.000000
mean,110.500000,1.523289,1.103023,1.353839,1.217928,1.030118,1.413396,-0.016890,1.108600,0.177273
std,63.652704,2.828754,3.453701,3.903195,1.223330,0.354056,3.383294,0.128853,1.018647,0.382770
min,1.000000,0.015096,-20.811849,-4.712397,0.027685,0.068823,0.092890,-0.592258,0.000000,0.000000
25%,55.750000,0.917511,0.932569,0.785517,0.962646,0.930413,0.899463,-0.071370,0.913937,0.000000
50%,110.500000,1.044853,1.000000,1.015498,1.084642,1.002396,1.011575,-0.014814,1.007465,0.000000
75%,165.250000,1.300227,1.094988,1.192857,1.243118,1.087991,1.204553,0.028612,1.108551,0.000000
max,220.000000,36.291185,46.466667,52.886734,13.081433,4.229119,49.301781,0.717842,13.058559,1.000000


In [6]:
df.median()

C:\Users\Avi\AppData\Local\Temp\ipykernel_21560\530051474.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df.median()


Company Name    110.500000
DSRI              1.044853
GMI               1.000000
AQI               1.015498
SGI               1.084642
DEPI              1.002396
SGAI              1.011575
ACCR             -0.014814
LEVI              1.007465
MANIPULATOR       0.000000
dtype: float64

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220 entries, 0 to 219
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Company Name  220 non-null    int64  
 1   Year Ending   220 non-null    object 
 2   DSRI          220 non-null    float64
 3   GMI           220 non-null    float64
 4   AQI           220 non-null    float64
 5   SGI           220 non-null    float64
 6   DEPI          220 non-null    float64
 7   SGAI          220 non-null    float64
 8   ACCR          220 non-null    float64
 9   LEVI          220 non-null    float64
 10  MANIPULATOR   220 non-null    int64  
dtypes: float64(8), int64(2), object(1)
memory usage: 19.0+ KB


In [8]:
df.MANIPULATOR.value_counts()

0    181
1     39
Name: MANIPULATOR, dtype: int64

> Taget variable is unbalanced 

> workflow:
1. Logistic regression model for baseline
2. scale the data
3. SVM and k-NN
4. Decision tree model

> Logistic regression model

In [9]:
X = df.drop(['Company Name','Year Ending','MANIPULATOR'], axis=1)
y = df.MANIPULATOR

# split the datasey into trina and test
X_trn, X_tst, y_trn, y_tst = train_test_split(X,y, 
                                              test_size=0.2,
                                              random_state=42)

# build logistic regression model
lm = sm.Logit(y_trn, sm.add_constant(X_trn)).fit()
lm.summary2()


Optimization terminated successfully.
         Current function value: 0.285326
         Iterations 9


<class 'statsmodels.iolib.summary2.Summary'>
"""
                         Results: Logit
=================================================================
Model:              Logit            Pseudo R-squared: 0.409     
Dependent Variable: MANIPULATOR      AIC:              118.4349  
Date:               2022-05-27 18:19 BIC:              146.9693  
No. Observations:   176              Log-Likelihood:   -50.217   
Df Model:           8                LL-Null:          -84.934   
Df Residuals:       167              LLR p-value:      6.3743e-12
Converged:          1.0000           Scale:            1.0000    
No. Iterations:     9.0000                                       
-------------------------------------------------------------------
          Coef.    Std.Err.      z      P>|z|     [0.025     0.975]
-------------------------------------------------------------------
const    -8.7195     1.8317   -4.7603   0.0000   -12.3095   -5.1294
DSRI      1.1200     0.4129    2.7123   0.0067     0.3107    1.9293
GMI       1.3670     0.4326    3.1603   0.0016     0.5192    2.2148
AQI       0.5429     0.1390    3.9054   0.0001     0.2704    0.8153
SGI       1.9698     0.6559    3.0032   0.0027     0.6842    3.2553
DEPI      0.9152     0.6984    1.3104   0.1901    -0.4537    2.2840
SGAI      0.2064     0.4273    0.4831   0.6290    -0.6310    1.0439
ACCR      8.5025     2.1812    3.8981   0.0001     4.2274   12.7775
LEVI     -0.3283     0.4345   -0.7556   0.4499    -1.1799    0.5233
=================================================================

"""

> from this model we can see that except 'DEPI','SGAI' AND 'LEVI' all others are statistically significant varialbes as p < 0.05

In [10]:
# test on test data
predy = lm.predict(sm.add_constant(X_tst))

test_df = pd.DataFrame({"actual":y_tst,
                        "Pred": predy}
                        )
test_df['final_pred'] = test_df.Pred.map(lambda x: 1 if x > 0.5 else 0)
test_df.head(10)

,actual,Pred,final_pred
132,0,0.006239,0
148,0,0.015190,0
93,0,0.031268,0
180,0,0.094016,0
15,1,0.584389,1
115,0,0.515787,1
172,0,0.034867,0
209,0,0.045111,0
75,0,0.632601,1
142,0,0.046117,0


In [11]:
# check the accuracy and scoring
accuracy = accuracy_score(test_df['actual'], test_df['final_pred'])
print("Accuracy of the logit model is: ", accuracy)
print("Roc_auc_score of logit model: ", roc_auc_score(y_tst, test_df.final_pred))

Accuracy of the logit model is:  0.8636363636363636
Roc_auc_score of logit model:  0.7105263157894737


> Support vector machine classification model

In [12]:
# scale the data 
scaler = MinMaxScaler()
scaler.fit(X_trn)
X_tr = scaler.transform(X_trn)
X_ts = scaler.transform(X_tst)

# build SVM model
svmc = SVC(C=0.01)
svmc.fit(X_tr,y_trn)
pred_y = svmc.predict(X_ts)
score = svmc.score(X_ts,y_tst)
accuracys = accuracy_score(y_tst,pred_y)
print("accuracy of the SVM model is: ", accuracys)
print("Roc_auc_score of SVM model: ", roc_auc_score(y_tst, pred_y))

accuracy of the SVM model is:  0.8636363636363636
Roc_auc_score of SVM model:  0.5


> k-nearest neighbors model

In [13]:
# build k-NN model

knn = KNeighborsClassifier(n_neighbors=3, weights="uniform", p=4)
knn.fit(X_tr,y_trn)
pre = knn.predict(X_ts)
acc = accuracy_score(y_tst,pre)
print("Accuracy of kNN model is:", acc)
print("ROC-AUC score is: ", roc_auc_score(y_tst, pre))

Accuracy of kNN model is: 0.9318181818181818
ROC-AUC score is:  0.75


> Decision tree model

In [17]:
# build Decsion tree model

Dt = DecisionTreeClassifier(criterion='gini', max_depth=7, max_leaf_nodes=3)
Dt.fit(X_tr, y_trn)
preds = Dt.predict(X_ts)
accu = accuracy_score(y_tst, preds)
print("Accuracy of Decision Tree model is:", accu)
print("ROC-AUC score is: ", roc_auc_score(y_tst, preds))

Accuracy of Decision Tree model is: 0.9545454545454546
ROC-AUC score is:  0.8333333333333333


# Conclusions:

> We build 4 different models to experiment and check performance of models with unbalanced dataset.

> It is found that with Logistics regression model we could archive upto 86% of accuracy with unbalanced class dataset and ROC-AUC being 71%

> It is found that with SVM model we archived 86% accuracy but ROC_auc was only 50%. this indicates that True postives are not predicted well they are predicted as false negatives.

> Our k-NN model performed well in terms of accuracy with 93% but ROC_AUC was lower which is not good model as well

> In our decision tree model, we archived best accuracy among all the models of 95% and highest ROC_AUC score as well. this is says that tree based models are more good learners for classification problems irrespective of balanced or unbalanced data.